# Database preparation

In [55]:
databaseTable3 = [
    {
        'tid': 1,
        'items': ['a', 'b', 'd', 'e', 'f', 'g'], 
        'quantities': [2, 2, 1, 3, 2, 1], 
        'profits': [-2, 1, 4, 1, -1, -2]},
    {
        'tid': 2, 
        'items': ['b', 'c'], 
        'quantities': [1, 5], 
        'profits': [-1, 1]},
    {
        'tid': 3, 
        'items': ['b', 'c', 'd', 'e', 'f'], 
        'quantities': [2, 1, 3, 2, 1], 
        'profits': [-1, 1, 4, 1, -1]},
    {
        'tid': 4, 
        'items': ['c', 'd', 'e'],
        'quantities': [2, 1, 3], 
        'profits': [1, 4, 1]},
    {
        'tid': 5, 
        'items': ['a', 'f'], 
        'quantities': [2, 3], 
        'profits': [2, -1]},
    {
        'tid': 6, 
        'items': ['a', 'b', 'c', 'd', 'e', 'f', 'g'], 
        'quantities': [2, 1, 4, 2, 1, 3, 1], 
        'profits': [1, 1, 1, 4, 1, -1, -2]},
    {
        'tid': 7, 
        'items': ['b', 'c', 'e'], 
        'quantities': [3, 2, 2], 
        'profits': [1, 2, 2]}
]

# Proposed TK_EMHUN algorithm

In [56]:
class TKEMHUN:
    def __init__(self, database: list, top_k: int):
        self.top_k = top_k
        self.database = database
        self.positive_items, self.negative_items, self.hybrid_items = self.classify_items(self.database)
        self.RTWU = self.calculate_RTWU(databaseTable3, self.positive_items, self.hybrid_items)
        self.RTWU_all_items = self.calculate_RTWU_all_items(self.database)
        self.minU = 25
        self.secondary = self.find_secondary(self.RTWU, self.minU)
        self.sorted_secondary, self.sorted_negative_items = self.sort_items_in_second_ni(self.secondary, self.negative_items, self.positive_items, self.hybrid_items, self.RTWU_all_items)

        self.pruned_database = self.prune_transactions(self.database, self.sorted_secondary, self.sorted_negative_items)
        self.sorted_pruned_database = self.sort_items_in_transactions(self.pruned_database, self.sorted_secondary, self.sorted_negative_items)
        self.final_sorted_database = self.sort_based_on_Def13(self.sorted_pruned_database, self.sorted_secondary, self.sorted_negative_items)

        self.RSU_values = self.calculate_RSU(self.final_sorted_database, self.sorted_secondary)
        self.primary = self.determine_Primary(self.RSU_values, self.minU)

    def classify_items(self, database: list):
        positive_items = set()
        negative_items = set()
        hybrid_items = set()
        for transaction in database:
            for item, profit in zip(transaction['items'], transaction['profits']):
                if profit > 0:
                    positive_items.add(item)
                elif profit < 0:
                    negative_items.add(item)
    
        hybrid_items = positive_items.intersection(negative_items)
        positive_items -= hybrid_items
        negative_items -= hybrid_items
        
        return list(positive_items), list(negative_items), list(hybrid_items)

    def determine_Primary(self, RSU, minU):
        return [item for item, utility in RSU.items() if utility >= minU]

    def find_secondary(self, RTWU, minU):
        Secondary = []
        for item, value in RTWU.items():
            if value >= minU:
                Secondary.append(item)
        return Secondary

    def sort_items_in_second_ni(self, Secondary, negative_items, positive_items, hybrid_items, RTWU):
        positive_secondary = [item for item in Secondary if item in positive_items]
        hybrid_secondary = [item for item in Secondary if item in hybrid_items]
        negative_secondary = [item for item in Secondary if item in negative_items]
        
        negative_only = [item for item in negative_items if item not in Secondary]
        
        positive_secondary.sort(key=lambda x: RTWU.get(x, 0))
        hybrid_secondary.sort(key=lambda x: RTWU.get(x, 0))
        negative_secondary.sort(key=lambda x: RTWU.get(x, 0))
        negative_only.sort(key=lambda x: RTWU.get(x, 0))
        
        sorted_secondary = positive_secondary + hybrid_secondary + negative_secondary
        sorted_negative = negative_only
        
        return sorted_secondary, sorted_negative
    
    def calculate_RSU(self, database, itemset):
        RSU = {item: 0 for item in itemset}

        for transaction in database:
            items = transaction['items']
            profits = transaction['profits']
            quantities = transaction.get('quantities', [1] * len(items))
            
            for index, item in enumerate(items):
                if item in itemset:
                    # utility of item X
                    u_item = profits[index] * quantities[index]
                    
                    # remaining relevant utility (rru)
                    rru = sum(
                        profits[i] * quantities[i]
                        for i in range(index + 1, len(items))
                        if profits[i] > 0
                    )

                    RSU[item] += u_item + rru
                    RSU[item] += transaction.get("u_project", 00)
            
        return RSU

    def calculate_RTU(self, transaction):
        RTU = 0
        for profit, quantity in zip(transaction['profits'], transaction['quantities']):
            if profit > 0:
                RTU += profit * quantity
        return RTU

    # RTWU without negative items
    def calculate_RTWU(self, database, positive_items, hybrid_items):
        RTWU = {}
        
        for transaction in database:
            RTU = self.calculate_RTU(transaction)
            for item in transaction['items']:
                if item in positive_items or item in hybrid_items :
                    if item not in RTWU:
                        RTWU[item] = 0
                    RTWU[item] += RTU
        
        return RTWU

    # RTWU including negative items
    def calculate_RTWU_all_items(self, database):
        RTWU = {}
        for transaction in database:
            RTU = self.calculate_RTU(transaction)
            for item in transaction['items']:
                if item not in RTWU:
                    RTWU[item] = 0
                RTWU[item] += RTU
        
        return RTWU

    def calculate_utility(self, itemset, database):
        utility = 0
        for transaction in database:
            if all(item in transaction['items'] for item in itemset):
                indices = [transaction['items'].index(item) for item in itemset if item in transaction['items']]
                quantities = transaction.get('quantities', [1] * len(transaction['items']))
                utility += sum(transaction['profits'][index] * quantities[index] for index in indices)
        return utility


    def project_database(self, itemset, database):
        print(f"***************projecting database***********")
        print(f"itemset: {itemset}")
        print(f"database: {database}")
        projected_db = []
        u_beta = 0
        for transaction in database:
            if itemset[-1] in transaction['items']:
                last_index = transaction['items'].index(itemset[-1]) 
                projected_items = transaction['items'][last_index + 1:]
                projected_profits = transaction['profits'][last_index + 1:]

                projected_quantities = (
                    transaction.get('quantities', [1] * len(transaction['items']))[last_index + 1:]
                )

                tid = transaction['tid']
                u_project = transaction['quantities'][last_index]*transaction['profits'][last_index]  + transaction.get("u_project", 00)
                u_beta += u_project
                if projected_items:
                    projected_db.append({
                        'tid': tid,
                        'items': projected_items,
                        'profits': projected_profits,
                        'quantities': projected_quantities,
                        'u_project': u_project
                    })
                    
        return projected_db, u_beta

    def calculate_RLU(self, database, itemset):
        RLU = {item: 0 for item in itemset}

        for transaction in database:
            for index, item in enumerate(transaction['items']):
                if item in itemset:
                    RLU[item] += transaction.get("u_project", 00) 
                    RLU[item] += sum(transaction['profits'][index] * transaction['quantities'][index] for index in range(len(transaction['profits'])) if transaction['profits'][index] > 0)
        
        return RLU

    def sort_items_in_transactions(self, pruned_database, sorted_secondary, sorted_negative_items):
        sorted_database = []
        combined_order = sorted_secondary + sorted_negative_items 
        
        for transaction in pruned_database:
            sorted_items = sorted(
                transaction['items'],
                key=lambda x: combined_order.index(x) if x in combined_order else float('inf')
            )
            
            sorted_quantities = [quantity for _, quantity in sorted(zip(transaction['items'], transaction['quantities']), key=lambda x: combined_order.index(x[0]))]
            sorted_profits = [profit for _, profit in sorted(zip(transaction['items'], transaction['profits']), key=lambda x: combined_order.index(x[0]))]
            
            sorted_transaction = {
                'tid': transaction['tid'],
                'items': sorted_items,
                'quantities': sorted_quantities,
                'profits': sorted_profits
            }
            sorted_database.append(sorted_transaction)
        
        return sorted_database
    
    def prune_transactions(self, database, sorted_secondary, sorted_negative_items):
        pruned_database = []
        
        for transaction in database:
            pruned_items = []
            pruned_quantities = []
            pruned_profits = []
            
            for item, quantity, profit in zip(transaction['items'], transaction['quantities'], transaction['profits']):
                if item in sorted_secondary or item in sorted_negative_items:
                    pruned_items.append(item)
                    pruned_quantities.append(quantity)
                    pruned_profits.append(profit)
            
            if pruned_items:
                pruned_transaction = {
                    'tid': transaction['tid'],
                    'items': pruned_items,
                    'quantities': pruned_quantities,
                    'profits': pruned_profits
                }
                pruned_database.append(pruned_transaction)
        
        return pruned_database
    
    def sort_based_on_Def13(self, sorted_pruned_database, sorted_secondary, sorted_negative_items):
        combined_order = sorted_secondary + sorted_negative_items

        def transaction_sort_key(transaction):
            items_order = [-combined_order.index(item) for item in transaction['items']]
            return items_order[::-1], len(transaction['items']), transaction['tid']

        sorted_database = sorted(
            sorted_pruned_database,
            key=transaction_sort_key
        )
        return sorted_database

    
    def run(self):
        return self.search(self.sorted_negative_items, [], self.final_sorted_database, self.primary, self.sorted_secondary, self.minU, [])

    # def search(self, eta, X, database, primary_items, secondary_items, minU, processing_top_k_pattern):
    #     print("**************search********************")
    #     for iter, i in enumerate(primary_items):
    #         beta = X + [i]
    #         print(f"\nProcessing item: {i}, Current Beta: {beta}")

    #         projected_db, u_beta = self.project_database(beta, database)
            
    #         print(f"Utility of Beta: {u_beta}")
    #         print(f"Projected Database: {projected_db}")

    #         index = 0
    #         while index < len(processing_top_k_pattern):
    #             if u_beta > processing_top_k_pattern[index][1]:
    #                 break
    #             index += 1

    #         processing_top_k_pattern.insert(index, (beta, u_beta))
    #         print("processing",processing_top_k_pattern)
    #         if len(processing_top_k_pattern) > self.top_k:
    #             processing_top_k_pattern = processing_top_k_pattern[:self.top_k]
    #             minU = processing_top_k_pattern[-1][1]

    #         if u_beta > minU:
    #             processing_top_k_pattern = self.searchN(eta, beta, projected_db, minU, processing_top_k_pattern)

    #         print("out if")
    #         print(f"projected_db: {projected_db}")
    #         rsu = self.calculate_RSU(projected_db, secondary_items[iter + 1:])
    #         rlu = self.calculate_RLU(projected_db, secondary_items[iter + 1:])
            
    #         print(f"RSU: {rsu}\nRLU: {rlu}")

    #         primary_beta = [z for z in secondary_items[iter + 1:] if rsu[z] >= minU]
    #         secondary_beta = [z for z in secondary_items[iter + 1:] if rlu[z] >= minU]

    #         print(f"Primary(β): {primary_beta}")
    #         print(f"Secondary(β): {secondary_beta}")

    #         processing_top_k_pattern = self.search(eta, beta, projected_db, primary_beta, secondary_beta, minU, processing_top_k_pattern)

    #     if len(processing_top_k_pattern) > self.top_k:
    #             processing_top_k_pattern = processing_top_k_pattern[:self.top_k]
    #     return processing_top_k_pattern
    
    def search(self, eta, X, database, primary_items, secondary_items, minU, processing_top_k_pattern):
        print("**************search********************")
        # Stack will store tuples of (eta, X, database, primary_items, secondary_items, iter)
        stack = [(eta, X, database, primary_items, secondary_items, 0)]
        
        while stack:
            eta, X, database, primary_items, secondary_items, iter = stack.pop()
            
            while iter < len(primary_items):
                i = primary_items[iter]
                beta = X + [i]
                print(f"\nProcessing item: {i}, Current Beta: {beta}")

                projected_db, u_beta = self.project_database(beta, database)
                print(f"Utility of Beta: {u_beta}")
                print(f"Projected Database: {projected_db}")

                # Insert into processing_top_k_pattern
                index = 0
                while index < len(processing_top_k_pattern):
                    if u_beta > processing_top_k_pattern[index][1]:
                        break
                    index += 1
                processing_top_k_pattern.insert(index, (beta, u_beta))
                print("processing", processing_top_k_pattern)

                if len(processing_top_k_pattern) > self.top_k:
                    processing_top_k_pattern = processing_top_k_pattern[:self.top_k]
                    minU = processing_top_k_pattern[-1][1]

                if u_beta > minU:
                    # Instead of recursive call, process searchN iteratively
                    processing_top_k_pattern = self.searchN(eta, beta, projected_db, minU, processing_top_k_pattern)

                print("out if")
                print(f"projected_db: {projected_db}")
                rsu = self.calculate_RSU(projected_db, secondary_items[iter + 1:])
                rlu = self.calculate_RLU(projected_db, secondary_items[iter + 1:])
                
                print(f"RSU: {rsu}\nRLU: {rlu}")

                primary_beta = [z for z in secondary_items[iter + 1:] if rsu[z] >= minU]
                secondary_beta = [z for z in secondary_items[iter + 1:] if rlu[z] >= minU]

                print(f"Primary(β): {primary_beta}")
                print(f"Secondary(β): {secondary_beta}")

                # Save current state before recursive call
                stack.append((eta, X, database, primary_items, secondary_items, iter + 1))
                
                # Set up for next iteration (equivalent to recursive call)
                eta = eta
                X = beta
                database = projected_db
                primary_items = primary_beta
                secondary_items = secondary_beta
                iter = 0

            if len(processing_top_k_pattern) > self.top_k:
                processing_top_k_pattern = processing_top_k_pattern[:self.top_k]
                
        return processing_top_k_pattern

    def searchN(self, eta, X, database, minU, n_processing_top_k_pattern):
        print("**************searchN********************")
        # Stack will store tuples of (eta, X, database, iter)
        stack = [(eta, X, database, 0)]
        
        while stack:
            eta, X, database, iter = stack.pop()
            
            while iter < len(eta):
                i = eta[iter]
                beta = X + [i]
                print(f"\nProcessing item: {i}, Current Beta: {beta}")

                projected_db, u_beta = self.project_database(beta, database)
                print(f"Database from Search: {database}")
                print(f"Utility of Beta: {u_beta}")
                print(f"Projected Database: {projected_db}")

                # Insert into n_processing_top_k_pattern
                index = 0
                while index < len(n_processing_top_k_pattern):
                    if u_beta > n_processing_top_k_pattern[index][1]:
                        break
                    index += 1
                n_processing_top_k_pattern.insert(index, (beta, u_beta))
                print("processing n", n_processing_top_k_pattern)

                if len(n_processing_top_k_pattern) > self.top_k:
                    n_processing_top_k_pattern = n_processing_top_k_pattern[:self.top_k]
                    minU = n_processing_top_k_pattern[-1][1]

                rsu = self.calculate_RSU(projected_db, eta[iter + 1:])
                print(f"RSU for Negative Items: {rsu}")

                primary_beta = [z for z in eta[iter + 1:] if rsu.get(z, 0) >= minU]
                print(f"Filtered Negative Items: {primary_beta}")

                # Save current state before recursive call
                stack.append((eta, X, database, iter + 1))
                
                # Set up for next iteration (equivalent to recursive call)
                eta = primary_beta
                X = beta
                database = projected_db
                iter = 0

        return n_processing_top_k_pattern

In [57]:
tk_emhun = TKEMHUN(database=databaseTable3, top_k=15)
result = tk_emhun.run()

**************search********************

Processing item: d, Current Beta: ['d']
***************projecting database***********
itemset: ['d']
database: [{'tid': 6, 'items': ['d', 'c', 'e', 'a', 'b', 'g', 'f'], 'quantities': [2, 4, 1, 2, 1, 1, 3], 'profits': [4, 1, 1, 1, 1, -2, -1]}, {'tid': 1, 'items': ['d', 'e', 'a', 'b', 'g', 'f'], 'quantities': [1, 3, 2, 2, 1, 2], 'profits': [4, 1, -2, 1, -2, -1]}, {'tid': 3, 'items': ['d', 'c', 'e', 'b', 'f'], 'quantities': [3, 1, 2, 2, 1], 'profits': [4, 1, 1, -1, -1]}, {'tid': 5, 'items': ['a', 'f'], 'quantities': [2, 3], 'profits': [2, -1]}, {'tid': 7, 'items': ['c', 'e', 'b'], 'quantities': [2, 2, 3], 'profits': [2, 2, 1]}, {'tid': 2, 'items': ['c', 'b'], 'quantities': [5, 1], 'profits': [1, -1]}, {'tid': 4, 'items': ['d', 'c', 'e'], 'quantities': [1, 2, 3], 'profits': [4, 1, 1]}]
Utility of Beta: 28
Projected Database: [{'tid': 6, 'items': ['c', 'e', 'a', 'b', 'g', 'f'], 'profits': [1, 1, 1, 1, -2, -1], 'quantities': [4, 1, 2, 1, 1, 3], 'u_pr

In [58]:
print(result)

[(['d', 'c', 'e'], 37), (['d', 'e'], 37), (['d', 'c'], 31), (['d', 'e', 'b'], 31), (['d'], 28), (['d', 'c', 'e', 'b'], 27), (['d', 'e', 'b', 'f'], 25), (['d', 'b'], 25), (['d', 'c', 'e', 'f'], 24), (['d', 'e', 'f'], 24), (['d', 'c', 'e', 'b', 'f'], 23), (['d', 'c', 'f'], 21), (['c', 'e'], 21), (['d', 'b', 'f'], 19), (['d', 'f'], 18)]
